# Checking enviroment states and action sets

In [1]:
import sys
import os.path
sys.path.append("..")
from PPO_Agent_with_intrinsic_reward import * #PPO_Agent_v2  PPO_Agent_with_Guided_AC
from ENV_DETAILS import *

2023-12-15 16:57:36.678700: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num devices available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
if os.name == 'nt':
    main_hyper_dir = "D:\\Artificial_Intelligence\\Portfolio\\RL_updated\\BipedalWalker\\" # Windows
    conda_python_exec = 'C:\\Users\\yanie\\anaconda3\\envs\\ai_dev\\python.exe '# Windows
    logs_dir = main_hyper_dir+'Hyperparam_tuning\\' # Windows
else:
    main_hyper_dir = "/media/n/NewDisk/Artificial_Intelligence/Portfolio/RL_updated/BipedalWalker/" # Linux
    conda_python_exec = '/home/n/anaconda3/envs/ai_dev/bin/python '# Linux
    logs_dir = main_hyper_dir+'Hyperparam_tuning_ppo/' # Linux

ENV = "BipedalWalker-v3"
SUCESS_CRITERIA_VALUE = ENV_DETAILS[ENV]["SUCESS_CRITERIA_VALUE"]
SUCESS_CRITERIA_EPOCH = ENV_DETAILS[ENV]["SUCESS_CRITERIA_EPOCH"]
EPISODES = ENV_DETAILS[ENV]["EPISODES"]

In [3]:

env = gym.make(ENV)
env


<TimeLimit<OrderEnforcing<BipedalWalker<BipedalWalker-v3>>>>

In [4]:
s = env.observation_space.sample()
s

array([ 2.06142634e-01,  3.36703509e-01,  5.60719927e-04, -6.13951385e-01,
        1.55297136e+00,  1.76780283e-01,  2.55007535e-01, -7.31230080e-01,
       -5.47144413e-01, -2.16561854e-02,  5.51912904e-01,  3.63126211e-02,
       -5.71376801e-01,  1.64752567e+00,  3.83927703e-01,  8.18679407e-02,
       -1.04001856e+00,  1.82784185e-01,  2.03154445e+00,  2.79704285e+00,
        9.48893070e-01, -7.42490232e-01,  6.32976472e-01, -1.12894274e-01],
      dtype=float32)

In [5]:
env.observation_space, env.observation_space.shape

(Box(-inf, inf, (24,), float32), (24,))

In [6]:
s = env.reset()
s

array([ 2.7474037e-03, -1.3443230e-05,  1.0457467e-03, -1.5999908e-02,
        9.1964334e-02, -1.3800329e-03,  8.6026824e-01,  2.4227477e-03,
        1.0000000e+00,  3.2371152e-02, -1.3799301e-03,  8.5381544e-01,
        9.7596686e-04,  1.0000000e+00,  4.4081402e-01,  4.4582012e-01,
        4.6142277e-01,  4.8955020e-01,  5.3410280e-01,  6.0246104e-01,
        7.0914888e-01,  8.8593185e-01,  1.0000000e+00,  1.0000000e+00],
      dtype=float32)

In [7]:
s = env.reset()
env.action_space, env.action_space, env.step(s)

(Box(-1.0, 1.0, (4,), float32),
 Box(-1.0, 1.0, (4,), float32),
 (array([-5.37584489e-03, -2.29245634e-04, -4.76412056e-03, -1.83619815e-03,
          4.37455863e-01,  8.13077167e-02,  1.12012625e-01, -9.95499268e-02,
          1.00000000e+00,  3.40303123e-01,  1.19120017e-01,  1.13705158e-01,
         -1.50200009e-01,  1.00000000e+00,  4.49633121e-01,  4.54739392e-01,
          4.70654190e-01,  4.99344319e-01,  5.44788301e-01,  6.14514112e-01,
          7.23336399e-01,  9.03656125e-01,  1.00000000e+00,  1.00000000e+00],
        dtype=float32),
  -0.37337466075876635,
  False,
  {}))

### Training/Hyperparam run

In [8]:
#sudo rm /usr/lib/python3.11/EXTERNALLY-MANAGED

%tensorboard --logdir logs_general/hyper

UsageError: Line magic function `%tensorboard` not found.


In [9]:
TUNING_TYPE = "BAYES"
HYPERPARAM_TUNING = True
writer= "Training/fit_PPO/"

In [10]:
if HYPERPARAM_TUNING:

    dir = r"Hyperparam_kt_ppo"
    project_name = "keras_tunning_ppo"
    

    tuner = kt.BayesianOptimization(
            MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/ppo/", evaluation_epoch = env._max_episode_steps, training_steps = 700000,
                sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value= SUCESS_CRITERIA_VALUE,
                discount_min = 0.95, discount_max = 0.99,
                gae_min = 0.93, gae_max = 0.98,
                lr_actor_min = 0.00001, lr_actor_max = 0.001,
                lr_critic_min = 0.00001, lr_critic_max = 0.001,
                entropy_factor_min = 0.001, entropy_factor_max = 0.1,
                dense_min = 32, dense_max = 256,
                environment_name=ENV,
                reward_scaler = 1, num_layers_act = 2, num_layers_crit =2, training_epoch = 200, memory_size=  env._max_episode_steps),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 50, 
            directory=dir,
            project_name=project_name
        )
    tuner.search(x=[0], y=[1])
else : 
    
        print("Acquiring parameters ....")
        writer= "logs_hyper/ppo/V0_"#Training/fit_ppo/"

        training_steps = 1000000
        entropy_factor = 0.05
        discount = 0.99
        dense_units_actor = [256,256]#64, 32]
        num_layers_actor = 2
        dense_units_critic = [256,256]#64,32]
        num_layers_crit =2
        num_layer_m = 1
        dense_units_model = [64]
        model = run_training(training_steps,  discount, dense_units_actor,  dense_units_critic, dense_units_model,  num_layers_actor, num_layers_crit, num_layer_m, writer, 
                      environment_name = ENV,reward_scaler = 1, return_agent = True, lr_actor= 0.00001, lr_critic= 0.00001, 
                      gae_lambda = 0.95, entropy_coeff = entropy_factor, policy_clip = 0.2, training_epoch = 50, id = 1, scaling_factor_reward = 0.1)
 
 

SyntaxError: invalid syntax (2457236999.py, line 17)

In [ ]:
exploration_tech = "soft"
hyperparam_combination=[]
for trials in tuner.oracle.get_best_trials(num_trials=10):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

In [ ]:
val_env = gym.make(ENV)#, render_mode = "rgb_array"
dir = r"Hyperparam_kt_ppo"

for trials in tuner.oracle.get_best_trials(num_trials=50):
    if int(trials.trial_id) in [6,11,15,16,23,24,26]:
        print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)    
        training_steps = 2000000
        entropy_factor = trials.hyperparameters.values["entropy_coeff"]
        discount = trials.hyperparameters.values["discount"]
        n_dense_layers = trials.hyperparameters.values["n_dense_layers"]


        dense_units =  [trials.hyperparameters.values['dense_units_'+str(i)] for i in range(n_dense_layers)]
        gae = trials.hyperparameters.values["gae_lambda"]
        lr = trials.hyperparameters.values["learning_rate"]
        policy_clip = trials.hyperparameters.values["policy_clip"]

        model = run_training(training_steps,  discount = discount, dense_units = dense_units, num_layers = n_dense_layers, writer = writer, 
                        environment_name = ENV,reward_scaler = 1, return_agent = True, lr= lr, sucess_criteria_epochs =SUCESS_CRITERIA_EPOCH, sucess_criteria_value = SUCESS_CRITERIA_VALUE,
                        evaluation_epoch = env._max_episode_steps, gae_lambda = gae, entropy_coeff = entropy_factor, 
                        policy_clip = policy_clip, training_epoch = 50, memory_size= env._max_episode_steps, id = int(trials.trial_id))
        
final_rewards = final_evaluation(model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./ppo_"+exploration_tech+"_video.mp4", env= ENV)

In [ ]:
env_model = tuner.get_best_models()[0]
final_rewards = final_evaluation(env_model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./ppo_"+exploration_tech+"_video.mp4", env= ENV)

In [ ]:
print("Final mean reward '",exploration_tech,"':", np.mean(final_rewards))


In [ ]:
final_rewards